In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate

chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.1)




In [6]:
from langchain.schema import HumanMessage, AIMessage, SystemMessage

messages = [
    SystemMessage(content="You are a senior student advisor at AUT, you are to explain queries with 10 years of experience knowledge",
                  ),
    AIMessage(content=""),
    HumanMessage(content="How do I make a study application and how long does it take to get the offer of place? Also if student is enrolled into postgraduate diploma in health science in child health, how many points does the student need to complete?, tell me numbers even if its approximate",),

]
chat.predict_messages(messages)

AIMessage(content='To make a study application at AUT, you would need to visit the AUT website and navigate to the section for prospective students. There, you can find information on how to apply for your desired program. Typically, you would need to create an account on the AUT online application portal, fill out the application form, upload any required documents (such as academic transcripts, CV, and proof of English proficiency), and pay the application fee if applicable.\n\nThe time it takes to receive an offer of place can vary depending on the program you are applying for, the volume of applications being processed, and the time of year. In general, it can take anywhere from a few weeks to a couple of months to receive an offer of place after submitting your application.\n\nFor the postgraduate diploma in health science in child health at AUT, the program typically consists of 120 points. This means that students would need to complete a total of 120 points worth of courses to 

NameError: name 'messages' is not defined